## Is the Elliptic Curve valid?

In [2]:
def is_valid_elliptic_curve(a, b):
   ##result should be anything but zero 
    #discriminant
    discr = -16 * (4 * a**3 + 27 * b**2)
    
    #curve is valid if discriminant = 0
    return discr != 0

def main():
    #inputs
    print("Enter coefficients 'A' and 'B' ")

    n = float(input("Enter coefficient A: "))
    
    m = float(input("Enter coefficient B: "))
    
                # Check if the curve is valid or not
    if is_valid_elliptic_curve(n, m):

        print(f"The curve is valid.")
    else:

        print(f"The curve is invalid.")

if __name__ == "__main__":
    main()


Enter coefficients 'A' and 'B' 
The curve is valid.


## Checking if point is on curve


In [3]:
def point_on_curve(x, y, a, b):
    
    leftSide = y**2
    right_side = x**3 + a*x + b

    return leftSide == right_side

def getInput():
    
    print("Enter coefficients of the elliptic curve:")
    ## if point_on_curve(x, y, x, y):
       ## print(f"The point ({x}, {y}) is on the curve.")
    ##else:
      ##  print(f"The point ({x}, {y}) is not on the curve.")

    x = float(input("Enter coefficient a:  "))
    y = float(input("Enter coefficient b:  "))
    print("enter coordinates for the point to check (x, y):  ")
    x = float(input("Enter x coordinate :  "))
    y = float(input("Enter y coordinate :  "))

    if point_on_curve(x, y, x, y):
        print(f"The point ({x}, {y}) is on the curve. ")
    else:
        print(f"The point ({x}, {y}) is not on the curve. ")

#to get input and check the point
getInput()
## curve -1, 1 -> (1,1) on the curve 

Enter coefficients of the elliptic curve:
enter coordinates for the point to check (x, y):  
The point (1.0, 3.0) is not on the curve. 


## ECC Using DH


In [4]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __repr__(self):
        return f"Point({self.x}, {self.y})"

def inverse_mod(k, p):
    # Extended Euclidean Algorithm to find modular inverse
    if k == 0:
        raise ZeroDivisionError('division by zero')

    if k < 0:
        return p - inverse_mod(-k, p)

    s, old_s = 0, 1
    t, old_t = 1, 0
    r, old_r = p, k

    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    gcd, x, y = old_r, old_s, old_t

    assert gcd == 1
    return x % p

def point_addition(p1, p2, a, p):
    if p1 is None:
        return p2
    if p2 is None:
        return p1

    if p1 == p2:
        if p1.y == 0:
            return None
        # Slope of the tangent line
        slope = (3 * p1.x ** 2 + a) * inverse_mod(2 * p1.y, p) % p
    else:
        if p1.x == p2.x:
            return None
        # Slope of the line through p1 and p2
        slope = (p2.y - p1.y) * inverse_mod(p2.x - p1.x, p) % p

    x3 = (slope ** 2 - p1.x - p2.x) % p
    y3 = (slope * (p1.x - x3) - p1.y) % p
    return Point(x3, y3)

def scalar_multiplication(base, k, a, p):
    result = None
    addend = base

    while k:
        if k & 1:
            result = point_addition(result, addend, a, p)
        addend = point_addition(addend, addend, a, p)
        k >>= 1

    return result

# Curve parameters y^2 = x^3 + ax + b over F_p
p = 97  # Example prime modulus
a = -1
b = 1
G = Point(3, 10)  # Base point (should be on the curve and chosen appropriately)

# Alice's private and public keys
d_A = 15
Q_A = scalar_multiplication(G, d_A, a, p)

# Bob's private and public keys
d_B = 30
Q_B = scalar_multiplication(G, d_B, a, p)

# Compute the shared secret
S_A = scalar_multiplication(Q_B, d_A, a, p)
S_B = scalar_multiplication(Q_A, d_B, a, p)

# Output the coordinates of the shared secret point
if S_A and S_B:
    print("Alice's Shared Secret:", (S_A.x, S_A.y))
    print("Bob's Shared Secret:", (S_B.x, S_B.y))
else:
    print("Error in computing the shared secret.")


Alice's Shared Secret: (45, 48)
Bob's Shared Secret: (45, 48)


## Encryption and Decryption using ECC (NIST PARAMETERS)

In [ ]:
import random

# Parameters for NIST P-256 curve
p = 2**256 - 2**224 + 2**192 + 2**96 - 1
a = p - 3
b = 41058363725152142129326129780047268409114441015993725554835256314039467401291
gx = 48439561293906451759052585252797914202762949526041747995844080717082404635286
gy = 36134250956749795798585127919587881956611106672985015071877198253568414405109
n = 115792089210356248762697446949407573529996955224135760342422259061068512044369

# Point at infinity
infinity = (None, None)

# Check if a point is on the curve
def is_on_curve(point):
    if point == infinity:
        return True
    x, y = point
    return (y**2 - x**3 - a*x - b) % p == 0

# Addition of points on the curve
def point_addition(point1, point2):
    if point1 == infinity:
        return point2
    if point2 == infinity:
        return point1
    x1, y1 = point1
    x2, y2 = point2
    if x1 == x2 and (y1 != y2 or y1 == 0):
        return infinity
    if x1 == x2:
        m = (3*x1**2 + a) * pow(2*y1, -1, p)
    else:
        m = (y1 - y2) * pow(x1 - x2, -1, p)
    x3 = (m**2 - x1 - x2) % p
    y3 = (m*(x1 - x3) - y1) % p
    return (x3, y3)

# Scalar multiplication of a point on the curve
def scalar_multiply(k, point):
    if k % n == 0 or point == infinity:
        return infinity
    result = infinity
    addend = point
    while k:
        if k & 1:
            result = point_addition(result, addend)
        addend = point_addition(addend, addend)
        k >>= 1
    return result

# Generate ECC key pair
def generate_key_pair():
    private_key = random.randint(1, n - 1)
    public_key = scalar_multiply(private_key, (gx, gy))
    return private_key, public_key

# Encrypt a message using ECC
def encrypt(message, receiver_public_key):
    k = random.randint(1, n - 1)
    x1, y1 = scalar_multiply(k, (gx, gy))
    x2, y2 = scalar_multiply(k, receiver_public_key)
    encrypted_message = []
    for char in message:
        encrypted_char = char ^ (x1 ^ x2)
        encrypted_message.append(encrypted_char)
    return encrypted_message, (x1, y1)

# Decrypt a message using ECC
def decrypt(encrypted_message, receiver_private_key, sender_public_key):
    x2, y2 = scalar_multiply(receiver_private_key, sender_public_key)
    decrypted_message = []
    for char in encrypted_message:
        decrypted_char = char ^ (sender_public_key[0] ^ x2)
        decrypted_message.append(decrypted_char)
    return decrypted_message

# Example usage
if __name__ == "__main__":
    # Generate key pair for sender
    sender_private_key, sender_public_key = generate_key_pair()

    # Generate key pair for receiver
    receiver_private_key, receiver_public_key = generate_key_pair()

    # Message to be encrypted
    message = b"Hello, this is a secret message haha!"

    # Encrypt the message using receiver's public key
    encrypted_message, ephemeral_key = encrypt(message, receiver_public_key)

    # Decrypt the message using receiver's private key
    decrypted_message = decrypt(encrypted_message, receiver_private_key, ephemeral_key)

    print("Original Message:", message)
    
    print("Encrypted Message:", encrypted_message[0])
    print("Decrypted Message:", bytes(decrypted_message))


Original Message: b'Hello, this is a secret message haha!'
Encrypted Message: 44676889780624636542605961342900600323271740497700067003387556917201399288368
Decrypted Message: b'Hello, this is a secret message haha!'
